In [4]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import torchvision
from voc import *
from coco import *
import torchvision.transforms as transforms
from torchvision.models import resnet152, resnet101, resnet18, resnet34, resnet50
from tqdm import tqdm
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import seed_everything
seed_everything(0)
name_map = {0: "resnet50", 1: "vit", 2: "swin"}

In [2]:
path_csv = '../data/coco'

from collections import defaultdict, Counter
class_num = defaultdict(int)
with open(path_csv + '/data/train_anno.json') as f:
  adj = np.zeros((80,80))
  import json
  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    for l in each['labels']:
      class_num[l] += 1

  nums = gt_labels.sum(axis=0)
  adj = []
  for i,col in enumerate(gt_labels.T):
    if i in [34]:
      print(i)
      selected = gt_labels[np.isin(col, [1.0]), :]
      nonzero_cnt = (selected != 0).sum(1)
      cnter = Counter(nonzero_cnt)
      print(cnter)
    cond_prob = gt_labels[np.isin(col,[1.0]),:].sum(axis=0)
    cond_prob[i] = 0
    adj.append(cond_prob)
    # print(adj[-1])
  nums = nums.tolist()
  nums.sort()
  nums.reverse()
  # nums = reversed(nums)
  print(max(nums), min(nums))
  di={'adj': np.asarray(adj), "nums": np.asarray(nums)}
  class_di = {k: v for k, v in sorted(class_num.items(), key=lambda item: item[1], reverse=True)} #sorted
print(class_di.keys(), class_di.values())

82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}
34
Counter({1: 1234, 2: 407, 3: 116, 4: 27, 5: 8, 8: 3, 7: 2, 6: 1})
45174.0 128.0
dict_keys([49, 22, 18, 27, 26, 13, 14, 36, 74, 2, 8, 12, 21, 57, 75, 24, 23, 41, 51, 28, 62, 72, 20, 16, 76, 68, 7, 32, 77, 58, 61, 42, 73, 44, 67, 65, 70, 9, 0, 10, 59, 50, 53, 11, 17, 37, 47, 5, 4, 34, 78, 52, 54, 64, 40, 3, 30, 33, 66, 39, 25, 15, 79, 45, 46, 63, 31, 19, 1, 60, 56, 43, 29, 38, 71, 55, 6, 48, 69, 35]) dict_values([45174, 8950, 8606, 8378, 6518, 5968, 5028, 4861, 4321, 3924, 3844, 3734, 3322, 3291, 3191, 3170, 3159, 3097, 3084, 3041, 2986, 2893, 2818, 2791, 2749, 2667, 2539, 2537, 2530, 2511, 2493, 2475, 2464, 2442, 2368, 2343, 2317, 2287, 2243, 2241, 2209, 2202, 2180, 2098, 2080, 2068, 2003, 1884, 1804, 1798, 1771, 1671, 1645, 1631, 1625, 1618, 1518, 1511, 1510, 1471, 1389, 1340, 1324, 1290, 1216, 1214, 1205, 1186, 1171, 1170, 1105, 1089, 1062, 821, 700, 673, 668, 481, 151, 128])


In [3]:
# test_dataset = COCO2014('../data/coco', phase='val', inp_name='../data/coco/coco_glove_word2vec.pkl')
# train_dataset = Voc2007Classification('data/voc', 'trainval', inp_name='data/voc/voc_glove_word2vec.pkl', LT=True)
# test_dataset = Voc2007Classification('data/voc', 'test', inp_name='data/voc/voc_glove_word2vec.pkl')
train_dataset = COCO2014('../data/coco', phase='train')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_dataset.transform = transforms.Compose([
                MultiScaleCrop(224, scales=(1.0, 0.875, 0.75, 0.66, 0.5), max_distort=2),
                transforms.ToTensor(),
                normalize,
            ])

test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=False)
from util import AveragePrecisionMeter
AP = AveragePrecisionMeter(difficult_examples=False)



[dataset] Done!
[annotation] Done!
[json] Done!


In [13]:
from models import *
from backbones.config import config
import pathlib

print(config)
for k, v in config.items():
  print("{} : {}".format(k, v))
  pathlib.Path('../figures/{}'.format(k)).mkdir(parents=True, exist_ok=True) 

m_li = [base_resnet50(model_path=config['resnet50'], num_classes=80, pretrained=True),\
   base_vit(config['vit'], 80, image_size=224, pretrained=True),\
      base_swin(config['swin'], 80, image_size=224, pretrained=True),\
         base_convnext(config['convnext'], 80, image_size=224, pretrained=True), \
           base_mlpmixer(config['mlpmixer'], num_classes=80, image_size=224, pretrained=True), \
             base_resnet101(model_path=config['resnet50'], num_classes=80, pretrained=True)
             ]
# m_li2 = [BaseResnet(m_li[0], 80)]
p_li = ['/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_resnet50_base_best.pth.tar', \
  '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_vit_base_best.pth.tar', \
    '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in22k-4-4-0_swin_base_best.pth.tar',\
  '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in22k-4-4-0_convnext_base_best.pth.tar',\
      '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_mlpmixer_base_best.pth.tar' ,\
  ]

def get_model(index):
  path = p_li[index]
  model = m_li[index]
  di = torch.load(path)
  print(di['best_score'])
  print(di.keys())
  model.load_state_dict(di['state_dict'])

  for n, p in model.named_parameters():
    if p.requires_grad==False:
      p.requires_grad=True
  return model


{'resnet50': 'resnetv2_50x3_bitm_in21k', 'vit': 'vit_base_patch16_224_in21k', 'swin': 'swin_base_patch4_window7_224_in22k', 'convnext': 'convnext_base_in22k', 'mlpmixer': 'mixer_b16_224_in21k'}
resnet50 : resnetv2_50x3_bitm_in21k
vit : vit_base_patch16_224_in21k
swin : swin_base_patch4_window7_224_in22k
convnext : convnext_base_in22k
mlpmixer : mixer_b16_224_in21k
Linear(in_features=768, out_features=80, bias=True)


Examples

In [ ]:
# pool with window of size=3, stride=2
m = nn.AvgPool1d(3, stride=2)
m(torch.tensor([[[1.,2,3,4,5,6,7]]])) #1,1,7

In [5]:
# pool of square window of size=3, stride=2
m = nn.AvgPool2d(3, stride=2)
# pool of non-square window
m = nn.AvgPool2d((63, 63), stride=(1, 1))
input = torch.randn(20, 768, 63, 63)
output = m(input)
output.shape

torch.Size([20, 768, 1, 1])

In [ ]:
decoder_layer = nn.TransformerDecoderLayer(d_model=768, nhead=1)
memory = torch.rand(32, 196, 768)
m = nn.AvgPool2d((63, 63), stride=(1, 1))
memory = m(memory)
memory = torch.squeeze(memory, -1)
memory = torch.squeeze(memory, -1)
tgt = torch.rand(32, 768)
out = decoder_layer(tgt, memory) #32,768

m = nn.AvgPool1d(196, stride=1)
out = m(out)
out.shape

In [ ]:

# Calculate symmetric padding for a convolution
def get_padding(kernel_size: int, stride: int = 1, dilation: int = 1, **_) -> int:
    padding = ((stride - 1) + dilation * (kernel_size - 1)) // 2
    return padding


In [8]:
# With square kernels and equal stride
m = nn.Conv2d(16, 33, 3, stride=2)
# # non-square kernels and unequal stride and with padding
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# # non-square kernels and unequal stride and with padding and dilation
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.randn(20, 16, 50, 50)
output = m(input)
output.shape

torch.Size([20, 33, 24, 24])

In [18]:
#convnext
model = get_model(3)
inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
out = model.features(inp)
print(out.shape) #[1, 1024, 7, 7])

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1][0], model.features[1][1]]) #3-3-27-3 blocks
out = seq(inp)
print(out.shape) #[1, 256, 28, 28]
b, n, h, w = out.shape

ap2 = nn.AvgPool2d((28-7+1, 28-7+1), stride=(1,1))
ap2(out)
ap1 = 


{'mAP': tensor(75.5266), 'OF1': 0.7372472199137282, 'CF1': 0.7008232877486551}
dict_keys(['epoch', 'arch', 'state_dict', 'best_score'])
torch.Size([1, 1024, 7, 7])
torch.Size([1, 256, 28, 28])


ValueError: too many values to unpack (expected 3)

In [11]:
#swin
model = get_model(2)
inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
out = model.features(inp)
print(out.shape)

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1], model.features[2][0], model.features[2][1]]) #2-2-17-2 blocks
out = seq(inp)
print(out.shape) #[1, 784, 256], [1, 196, 512]
b, n, h = out.shape
out = out.reshape((1, -1))

m = nn.AvgPool1d(n*h - 1024 + 1, stride=1)
out = m(out).squeeze(-1)
print(out.shape)

logit = model.fc(out)
print(logit.shape)

{'mAP': tensor(74.8108), 'OF1': 0.7193339297047961, 'CF1': 0.6759560773634149}
dict_keys(['epoch', 'arch', 'state_dict', 'best_score'])
torch.Size([1, 1024])
torch.Size([1, 196, 512])
torch.Size([1, 1024])
torch.Size([1, 80])


In [ ]:
#vit
model = get_model(1)
inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
out = model.features(inp)
print(out.shape)

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1], model.features[2][0], model.features[2][1], model.features[2][2]])
out = seq(inp)
print(out.shape) #([1, 196, 768])
b, n, h = out.shape

out = torch.swapaxes(out, 1,2)
m = nn.AvgPool1d(196, stride=1)
out = m(out).squeeze(-1)
print(out.shape)

logit = model.fc(out)
print(logit.shape)

In [10]:
#resnet101
model = get_model(0)
inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
out = model.features(inp)
out = model.head.global_pool(out)
print(out.shape) #torch.Size([1, 6144, 1, 1])

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1][0], model.features[1][1],model.features[1][2]])
out = seq(inp)
print(out.shape) #[1, 3072, 14, 14])
b, n, h, w = out.shape
out = out.reshape((1, -1))
print(out.shape)

m = nn.AvgPool1d(n*h*w - 6144 + 1, stride=1)
# m = nn.Conv2d(n, )
out = m(out)
out = out.reshape((-1, 6144, 1, 1))
print(out.shape)

logit = model.head.fc(out)
print(logit.shape)
logit = logit.flatten()
print(logit.shape)

NameError: name 'get_model' is not defined

mlpmixer

In [ ]:
model = get_model(4)
# import torch.nn.functional as F

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1][0]])
out = seq(inp)
print(out.shape)
out = torch.swapaxes(out, 1,2)
m = nn.AvgPool1d(196, stride=1)
out = m(out).squeeze(-1)
# out = torch.mean(out.view(out.size(0), out.size(1), -1), dim=2)
torch.mean(torch.stack([out, out]), 1)
print(out.shape)

logit = model.fc(out)
print(logit.shape)

In [ ]:
stages = model.features[1]
# print(stages[3].blocks[2].norm3)
target_layers = [stages[1].blocks[2].norm3]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model(0).to(device)
model = model.eval()
for i, (input, target) in tqdm(enumerate(test_loader)):
  img, path = input
  target[target == 0] = 1
  target[target == -1] = 0
  feat_Var = torch.autograd.Variable(img).float().to(device)
  
  # output = model(feat_Var, None).detach()
  output = model(feat_Var, None).detach()
  # print(output.requires_grad, target.requires_grad)
  # print(output.shape, target.shape)
  AP.add(output, target)

# map = 100 * AP.value().mean()
# print(100 * AP.value())
# ap_li = 100 * AP.value()
print(AP.scores.shape)